In [2]:
# -*- coding: utf-8 -*-
"""
Created on Tue May 10 14:31:55 2022

@author: HASEE
"""

import sys
sys.path.append('/content/drive/My Drive/Colab Notebooks/FCN')
sys.path.append('/content/drive/My Drive/Colab Notebooks/FCN/')

import random
import glob
import torch
from torch.utils.data import Dataset 
from torch.utils.data import DataLoader
from torch import optim 
import torch.nn as nn
import cv2 
import os
import glob
import numpy as np
import ntpath
import torch.nn as nn
import torch.nn.functional as F

import matplotlib.pyplot as plt # plt 用于显示图片
import matplotlib.image as mpimg # mpimg 用于读取图片

"""
  卷积： 卷积-BN-relu
             N
  L1:卷积-卷积-池化  N/2
  L2:卷积-卷积-池化  N/4
  L3:卷积-卷积-池化  N/8
  L4:卷积-卷积-池化  N/16
  L5:卷积-卷积-池化  N/32
             N/32
  L6:反卷积-卷积-卷积 N/16
  L7:反卷积-卷积-卷积 N/8
  L8:反卷积-卷积-卷积 N/4
  L9:反卷积-卷积-卷积 N/2
  L10:反卷积-卷积-卷积 N

  L11: 1*1卷积output N


  tips:
      1. 让卷积前后输入输出size相同：kernel_size = 3,padding = 1
      2. 2倍上卷积： kernel_size=2, stride=2
"""


'\n  卷积： 卷积-BN-relu\n             N\n  L1:卷积-卷积-池化  N/2\n  L2:卷积-卷积-池化  N/4\n  L3:卷积-卷积-池化  N/8\n  L4:卷积-卷积-池化  N/16\n  L5:卷积-卷积-池化  N/32\n             N/32\n  L6:反卷积-卷积-卷积 N/16\n  L7:反卷积-卷积-卷积 N/8\n  L8:反卷积-卷积-卷积 N/4\n  L9:反卷积-卷积-卷积 N/2\n  L10:反卷积-卷积-卷积 N\n\n  L11: 1*1卷积output N\n\n\n  tips:\n      1. 让卷积前后输入输出size相同：kernel_size = 3,padding = 1\n      2. 2倍上卷积： kernel_size=2, stride=2\n'

In [3]:


class double_conv2d(nn.Module) :#双卷积： （卷积-BN-RL,卷积-BN-RL） 未含池化操作
  def __init__(self,in_channel,out_channel):
    super().__init__()
    self.module = nn.Sequential(
        nn.Conv2d(in_channels = in_channel,out_channels = out_channel,kernel_size = 3,padding = 1),
        nn.BatchNorm2d(out_channel),
        nn.ReLU(),
        nn.Conv2d(in_channels = out_channel,out_channels = out_channel,kernel_size = 3,padding = 1),
        nn.BatchNorm2d(out_channel),
        nn.ReLU(),
    )
  def forward(self,x) :
    y = self.module(x)
    return y

class downsample(nn.Module) :
  def __init__(self,in_channel,out_channel) :
    super().__init__()
    self.module = nn.Sequential(
        double_conv2d(in_channel,out_channel),
        nn.MaxPool2d(2)
    )
  def forward(self,x) :
    y = self.module(x)
    return y


class upsample(nn.Module) :#上采样： (反卷积-双卷积)
  def __init__(self,in_channel,out_channel):
    super().__init__()
    self.upsample = nn.ConvTranspose2d(in_channel,in_channel,kernel_size = 2,stride = 2)
    self.double_conv2d = double_conv2d(in_channel,out_channel)

  def forward(self,x,x_target):#反卷积后得到新的x后，需要将新的x形状微调为x_target(maxpool前的size)，这是由于奇数尺寸maxpool后会有信息丢失
    
    x = self.upsample(x)
    deta_w = torch.tensor([x_target.size()[2] - x.size()[2] ])
    deta_h = torch.tensor([x_target.size()[3] - x.size()[3] ])
    pad = [deta_h // 2, deta_h - deta_h // 2,deta_w // 2, deta_w - deta_w //2 ]
    
    x = F.pad(x,pad)
    y = self.double_conv2d(x)
    return y 

class FCN(nn.Module) :
  def __init__(self,in_channel,out_channel):
    super().__init__()
    self.down1 = downsample(in_channel=in_channel,out_channel=64)
    self.down2 = downsample(in_channel=64,out_channel=128)
    self.down3 = downsample(in_channel=128,out_channel=256)
    self.down4 = downsample(in_channel=256,out_channel=512)
    self.down5 = downsample(in_channel=512,out_channel=1024)


    self.up1 = upsample(1024,512)
    self.up2 = upsample(512,256)
    self.up3 = upsample(256,128)
    self.up4 = upsample(128,64)
    self.up5 = upsample(64,out_channel)

    self.output = nn.Conv2d(out_channel,out_channel,kernel_size=1,stride=1,padding=0)
    

  def forward(self,x) :
    
    x1 = self.down1(x)   #N/2
    x2 = self.down2(x1)  #N/4
    x3 = self.down3(x2)  #N/8
    x4 = self.down4(x3)  #N/16
    x5 = self.down5(x4)  #N/32

    y1 = self.up1(x5,x4)  # 1/16
    y2 = self.up2(y1,x3)  # 1/8
    y3 = self.up3(y2,x2)  # 1/4
    y4 = self.up4(y3,x1)  # 1/2
    y5 = self.up5(y4,x)   # 1

    score = self.output(y5)
    return score


    
    
    

In [4]:

class data_loader(Dataset) :
  def __init__(self,path) :
    self.path = path
    self.dataset = glob.glob (os.path.join(path,"image/*.png"))

  def __getitem__(self,index) :
    image_path = self.dataset[index]
    label_path = image_path.replace("image","label")

    image = cv2.cvtColor( cv2.imread(image_path) , cv2.COLOR_BGR2GRAY )
    label = cv2.cvtColor( cv2.imread(label_path) , cv2.COLOR_BGR2GRAY )

    image = cv2.resize(image,(501,501))
    label = cv2.resize(label,(501,501))

    image = image.reshape(1,image.shape[0],image.shape[1])
    label = label.reshape(1,label.shape[0],label.shape[1])

    label[label > 0] = 1
    return image , label

  def __len__(self) :
    return len(self.dataset)



In [5]:

def train(net,path,device,lr,epoches,batch_size) :

  train_set = data_loader(path)
  train_loader = torch.utils.data.DataLoader(dataset=train_set , batch_size=batch_size, shuffle= True)

  optimizer = optim.Adam(params= net.parameters(), lr = lr)
  criterion = nn.BCEWithLogitsLoss()

  loss_y = []
  loss_x = []
  net = net.to(device)
  net.train()
  for epoch in range(epoches) :
    cost = 0
    optimizer.zero_grad()
    for image,label in train_loader :
      image = image.to(device, dtype=torch.float32)
      label = label.to(device, dtype=torch.float32)

      predict_map = net(image)
      loss = criterion(predict_map,label)
      cost = loss.item()

      loss.backward()
      optimizer.step()
    torch.save(net.state_dict(), path+'trained_model.pth')
    print("epoch ",epoch," loss = ",cost)
    loss_y.append(cost)
    loss_x.append(epoch)
  show_loss = plt.plot(loss_x,loss_y,'r--',label='loss')
  plt.title('loss conditions')
  plt.xlabel('epoch')
  plt.ylabel('loss')
  plt.show()
  


In [18]:
def test(net,device,trained_model_path,path) :

  test_set = glob.glob(os.path.join(path,"image/*.png"))
  net.to(device)
  net.load_state_dict(torch.load(trained_model_path,map_location=device))
  net.eval()

  print("len = ",len(test_set))
  for image_path in test_set :
    print(image_path)
    image = cv2.imread(image_path)
    image_gray =cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
    image_numpy = image_gray.reshape(1,1,image_gray.shape[0],image_gray.shape[1])
    image_tensor =torch.from_numpy(image_numpy).float()
    image_tensor = image_tensor.to(device, dtype=torch.float32)

    predict_map = net(image_tensor)
    predict_map = np.array(predict_map.data.cpu()[0])[0]
    predict_map[predict_map >= 0.5] = 255
    predict_map[predict_map < 0.5] = 0
    predict_map = predict_map.astype( np.uint8 )
    predict_map = cv2.Canny(predict_map, 0, 1)
    print(predict_map.shape)
    for i in range(predict_map.shape[0]) :
      for j in range(predict_map.shape[1]) :
        if (predict_map[i][j] == 255) :
          image[i][j] = [0,255,0]
          image[i][j+1] = [0,255,0]
          image[i][j-1] = [0,255,0]
        else :
          image[i][j] = image[i][j]

    save_path = image_path.split('.')[0]+"_result.png"
    cv2.imwrite(save_path,image)
    


    

In [ ]:
"""
train.py
"""
net = FCN(1,1)
path = "/content/drive/MyDrive/Colab Notebooks/FCN/data/data/train/"
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu') ; print(device)
lr = 1e-2
epoches = 512
batch_size = 3
train(net,path,device,lr,epoches,batch_size)

In [19]:
"""
test.py
"""
net = FCN(1,1)
trained_model_path = "/content/drive/MyDrive/Colab Notebooks/FCN/data/data/train/trained_model.pth"
path = "/content/drive/MyDrive/Colab Notebooks/FCN/data/data/test/"
test(net,device,trained_model_path,path)

len =  36
/content/drive/MyDrive/Colab Notebooks/FCN/data/data/test/image/0.png
(501, 501)


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:41: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').


/content/drive/MyDrive/Colab Notebooks/FCN/data/data/test/image/1.png
(501, 501)
/content/drive/MyDrive/Colab Notebooks/FCN/data/data/test/image/2.png
(501, 501)
/content/drive/MyDrive/Colab Notebooks/FCN/data/data/test/image/3.png
(501, 501)
/content/drive/MyDrive/Colab Notebooks/FCN/data/data/test/image/4.png
(501, 501)
/content/drive/MyDrive/Colab Notebooks/FCN/data/data/test/image/5.png
(501, 501)
/content/drive/MyDrive/Colab Notebooks/FCN/data/data/test/image/7.png
(501, 501)
/content/drive/MyDrive/Colab Notebooks/FCN/data/data/test/image/6.png
(501, 501)
/content/drive/MyDrive/Colab Notebooks/FCN/data/data/test/image/9.png
(501, 501)
/content/drive/MyDrive/Colab Notebooks/FCN/data/data/test/image/8.png
(501, 501)
/content/drive/MyDrive/Colab Notebooks/FCN/data/data/test/image/10.png
(501, 501)
/content/drive/MyDrive/Colab Notebooks/FCN/data/data/test/image/11.png
(501, 501)
/content/drive/MyDrive/Colab Notebooks/FCN/data/data/test/image/13.png
(501, 501)
/content/drive/MyDrive/Co